In [245]:
import pandas as pd
import numpy as np

In [246]:
def CDS_code_maker(CC, DC, SC):
    if len(str(int(CC)))<2:
        CC = '0'+ str(int(CC))
    else:
        CC = str(int(CC))
    DC = str(int(DC))
    if len(str(int(SC)))<7:
        SC = '0'*(7-len(str(int(SC))))+ str(int(SC))
    else:
        SC = str(int(SC))
    if len(CC+DC+SC) != 14:
        print(CC+DC+SC)
    return CC+DC+SC

In [247]:
path = 'CA_edu_data/'

In [248]:
numeric = ['CohortStudents', 'RegularHSGradCount',
       'RegularHSGradRate', 'MetUCCSUCount', 'MetUCCSURate',
       'SealOfBiliteracyCount', 'SealOfBiliteracyRate',
       'GoldenStateSealMeritCount', 'GoldenStateSealMeritRate',
       'CHSPECompleterCount', 'CHSPECompleterRate', 'AdultEdDiplomaCount',
       'AdultEdDiplomaRate', 'SPEDCertificateCount', 'SPEDCertificateRate',
       'GEDCompleterCount', 'GEDCompleterRate', 'OtherTransferCount',
       'OtherTransferRate', 'DropoutCount', 'DropoutRate',
       'StillEnrolledCount', 'StillEnrolledRate']

In [249]:
grad_rates = pd.read_table(path+'grad_rates_cohort1617.txt', sep = '\t')
grad_rates = grad_rates[(grad_rates.AggLevel == 'S') & (grad_rates.DASSYN == 'No')& (grad_rates.CharterYN != 'All')]
grad_rates['CDS'] = [CDS_code_maker(grad_rates.CountyCode.iloc[ik], grad_rates.DistrictCode.iloc[ik], grad_rates.SchoolCode.iloc[ik]) for ik in range(len(grad_rates))]
grad_rates.reset_index()
grad_rates.set_index('CDS')

for col in numeric:
    try: 
        grad_rates[col] = grad_rates[col].str.replace("*", "0")
    except:
        print('no dice')
    grad_rates[col] = pd.to_numeric(grad_rates[col])

In [250]:
schools = grad_rates.groupby('CDS')
repcats = grad_rates.groupby('ReportingCategory')

In [251]:
repcat_cols = ['CohortStudents', 'RegularHSGradCount','RegularHSGradRate', 'MetUCCSUCount', 'MetUCCSURate','DropoutCount', 'DropoutRate']
# gradmeta_cols = ['CountyName', 'DistrictName', 'SchoolName','CDS', 'CharterYN']

In [271]:
grad_rates2 = grad_rates[['CDS', 'CharterYN']].drop_duplicates()

for repcat in repcats:
    sub_df = repcat[1][['CDS']]
    sub_df.set_index(['CDS'])
    for col in repcat_cols:
        sub_df[col+ '_' + repcat[0]] = repcat[1][col]
    grad_rates2 = grad_rates2.join(sub_df.set_index(['CDS'], verify_integrity=True), on = ['CDS'], how = 'outer')

grad_rates2.reset_index()
grad_rates2 = grad_rates2.set_index(['CDS'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [272]:
def split_GS(grades_vect):

    def splitter(offered):
        try:
            [min_grade, max_grade] = offered.split('-')
        except:
            [min_grade, max_grade] = [np.nan, np.nan]
        return min_grade, max_grade

    min_grdes = []
    max_grdes = []
    for ik in range(len(grades_vect)):
        min_grade, max_grade = splitter(grades_vect[ik])
        min_grdes.append(min_grade)
        max_grdes.append(max_grade)
        
    return min_grdes, max_grdes

In [273]:
meta_cols = ['CDS', 'NCESDist', 'NCESSchool', 'StatusType', 'County', 'District',
       'School', 'City', 'Zip', 'OpenDate', 'ClosedDate', 'Charter', 'CharterNum',
       'FundingType', 'DOC', 'DOCType', 'SOC', 'SOCType', 'EdOpsCode',
       'EdOpsName', 'EILCode', 'EILName', 'GSoffered', 'GSserved', 'Virtual',
       'Magnet', 'Latitude', 'Longitude']

In [274]:
school_meta = pd.read_table(path + 'pubschls.txt', sep = '\t')
school_meta = school_meta.rename(columns={'CDSCode':'CDS'})# = school_meta
school_meta = school_meta[meta_cols]
school_meta.reset_index()
school_meta.set_index('CDS')

,NCESDist,NCESSchool,StatusType,County,District,School,City,Zip,OpenDate,ClosedDate,...,EdOpsCode,EdOpsName,EILCode,EILName,GSoffered,GSserved,Virtual,Magnet,Latitude,Longitude
CDS,,,,,,,,,,,,,,,,,,,,,
1100170000000,691051.0,NaN,Active,Alameda,Alameda County Office of Education,NaN,Hayward,94544-1136,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.658212,-122.09713
1100170109835,691051.0,10546.0,Closed,Alameda,Alameda County Office of Education,FAME Public Charter,Newark,94560-5359,08/29/2005,07/31/2015,...,TRAD,Traditional,ELEMHIGH,Elementary-High Combination,K-12,K-12,P,N,37.521436,-121.99391
1100170112607,691051.0,10947.0,Active,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,Oakland,94612-3355,08/28/2006,NaN,...,TRAD,Traditional,HS,High School,9-12,9-12,N,N,37.804520,-122.26815
1100170118489,691051.0,12283.0,Closed,Alameda,Alameda County Office of Education,Aspire California College Preparatory Academy,Berkeley,94703-1414,08/21/2008,06/30/2015,...,TRAD,Traditional,HS,High School,9-12,9-12,N,N,37.868991,-122.27844
1100170123968,691051.0,12844.0,Active,Alameda,Alameda County Office of Education,Community School for Creative Education,Oakland,94606-4903,08/22/2011,NaN,...,TRAD,Traditional,ELEM,Elementary,K-8,K-8,N,N,37.784648,-122.23863
1100170124172,691051.0,12901.0,Active,Alameda,Alameda County Office of Education,Yu Ming Charter,Oakland,94608-1265,08/09/2011,NaN,...,TRAD,Traditional,ELEM,Elementary,K-8,K-7,N,N,37.847375,-122.28356
1100170125567,691051.0,13008.0,Active,Alameda,Alameda County Office of Education,Urban Montessori Charter,Oakland,94619-3312,08/27/2012,NaN,...,TRAD,Traditional,ELEM,Elementary,K-8,K-7,N,N,37.778352,-122.18950
1100170130302,691051.0,NaN,Closed,Alameda,Alameda County Office of Education,"Technical, Agricultural & Nat.",Hayward,94541,07/01/1980,06/30/1989,...,NaN,NaN,UG,Ungraded,NaN,NaN,NaN,NaN,NaN,NaN
1100170130401,691051.0,9264.0,Active,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,San Leandro,94578-1005,07/01/1980,NaN,...,JUV,Juvenile Court School,HS,High School,K-12,7-12,N,N,37.712878,-122.11173


In [275]:
min_grades, max_grades = split_GS(school_meta.GSoffered)
school_meta['GSoffered_min'] = min_grades
school_meta['GSoffered_max'] = max_grades

min_grades, max_grades = split_GS(school_meta.GSserved)
school_meta['GSserved_min'] = min_grades
school_meta['GSserved_max'] = max_grades

In [306]:
HS = school_meta[(school_meta.EdOpsCode == 'TRAD') & 
                 (school_meta.EILCode == 'HS') & 
                 (school_meta.StatusType == 'Active')]
HS.reset_index()
HS = HS.set_index(['CDS'])


In [312]:
grad_rates2.describe()

,CohortStudents_GF,RegularHSGradCount_GF,RegularHSGradRate_GF,MetUCCSUCount_GF,MetUCCSURate_GF,DropoutCount_GF,DropoutRate_GF,CohortStudents_GM,RegularHSGradCount_GM,RegularHSGradRate_GM,...,MetUCCSURate_SS,DropoutCount_SS,DropoutRate_SS,CohortStudents_TA,RegularHSGradCount_TA,RegularHSGradRate_TA,MetUCCSUCount_TA,MetUCCSURate_TA,DropoutCount_TA,DropoutRate_TA
count,1796.000000,1796.000000,1796.000000,1796.000000,1793.000000,1796.000000,1796.000000,1845.000000,1845.000000,1845.000000,...,1814.000000,1823.000000,1823.000000,1869.000000,1869.000000,1869.000000,1869.000000,1861.000000,1869.000000,1869.000000
mean,115.663140,107.678174,74.165423,63.641425,45.313720,4.115813,4.487751,113.196748,100.723035,67.552412,...,36.732911,8.614920,6.513275,223.822365,203.505618,73.102836,108.204387,40.886298,10.359551,6.394489
std,111.161232,106.646826,36.506307,70.511229,32.232143,6.892486,8.625989,112.747942,104.414236,37.658741,...,29.594254,13.542057,10.489162,222.102935,209.517447,34.723552,125.792796,30.757350,15.555771,10.649359
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,16.000000,72.425000,1.000000,7.700000,0.000000,0.000000,17.000000,11.000000,47.600000,...,4.225000,0.000000,0.000000,33.000000,23.000000,67.900000,1.000000,7.700000,1.000000,0.300000
50%,73.000000,67.000000,92.900000,40.000000,50.900000,2.000000,1.500000,67.000000,56.000000,87.600000,...,37.450000,4.000000,3.000000,129.000000,112.000000,90.500000,64.000000,43.100000,5.000000,2.600000
75%,202.000000,190.000000,97.100000,103.000000,68.500000,6.000000,4.900000,205.000000,181.000000,94.500000,...,53.900000,11.000000,7.700000,398.000000,365.000000,95.600000,170.000000,61.500000,13.000000,7.300000
max,551.000000,517.000000,100.000000,435.000000,100.000000,134.000000,100.000000,544.000000,501.000000,100.000000,...,100.000000,179.000000,100.000000,1095.000000,1011.000000,100.000000,800.000000,100.000000,209.000000,100.000000


In [313]:
HS = HS.join(grad_rates2, how= 'left')
# HS2 = HS.merge(grad_rates2, right_on='CDS')#, right_on='CDS')

In [314]:
HS.CohortStudents_GF

CDS
1100170112607    NaN
1611190130229    NaN
1611190130625    NaN
1611270130450    NaN
1611430131177    NaN
1611500132225    NaN
1611500133876    NaN
1611680132746    NaN
1611760130062    NaN
1611760134270    NaN
1611760134452    NaN
1611760135244    NaN
1611760138693    NaN
1611920108670    NaN
1611920127944    NaN
1611920133629    NaN
1611920135319    NaN
1611920138339    NaN
1612000133397    NaN
1612000134536    NaN
1612340130054    NaN
1612420134668    NaN
1612590100065    NaN
1612590100701    NaN
1612590108944    NaN
1612590110189    NaN
1612590111856    NaN
1612590114868    NaN
1612590115238    NaN
1612590115386    NaN
                  ..
56725465634548   NaN
56725465634761   NaN
56726035630322   NaN
56726035635008   NaN
56726035635933   NaN
56726035636188   NaN
56726525630348   NaN
56726525630793   NaN
56726525637814   NaN
56726525637822   NaN
56737595630116   NaN
56737595633748   NaN
56737595637004   NaN
56738745630132   NaN
56739400121426   NaN
56739405630371   NaN
567394056

In [278]:
EL_numeric = ['KDGN', 'GR_1',
       'GR_2', 'GR_3', 'GR_4', 'GR_5', 'GR_6', 'GR_7', 'GR_8', 'GR_9', 'GR_10',
       'GR_11', 'GR_12', 'UNGR', 'TOTAL_EL']

In [279]:
EL = pd.read_table(path+ 'eng_learn1617.txt', sep = '\t')
for col in EL_numeric:
    EL[col] = pd.to_numeric(EL[col])
EL.reset_index()
EL = EL.set_index(['CDS'])



In [281]:
lang_cols = ['GR_9', 'GR_10', 'GR_11', 'GR_12']
langs = EL.groupby('LANGUAGE')

grad_rates3 = HS
for lang in langs:
    sub_df = repcat[1][['CDS']]
    sub_df.reset_index()
    sub_df = sub_df.set_index(['CDS'])
    for col in lang_cols:
        sub_df[col+ '_' + lang[0]] = lang[1][col]
    grad_rates3 = grad_rates3.join(sub_df, how= 'outer')#.set_index(['CDS'], verify_integrity=True), on = ['CDS'], how = 'outer')


/usr/local/lib/python3.6/site-packages/pandas/core/reshape/merge.py:736: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  sort=self.sort)
/usr/local/lib/python3.6/site-packages/pandas/core/reshape/merge.py:736: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  sort=self.sort)


In [296]:
grad_rates2.CohortStudents_GF

CDS
01100170112607     37.0
01611190130609     27.0
01611190130229    219.0
01611190106401     22.0
01611190130625     20.0
01611190132142    118.0
01611190119222     11.0
01611190000001      NaN
01611270130450    147.0
01611270000001      NaN
01611430131177    412.0
01611430000001      0.0
01611430122697     30.0
01316090131755      0.0
01316170131763     21.0
01611500132225    360.0
01611500133876      0.0
01611500000001      NaN
01750930132704    238.0
01611680132746     14.0
01611760130062    246.0
01611760130534     30.0
01611760134270    252.0
01611760134452    157.0
01611760135244    216.0
01611760000001      0.0
01611760130435     11.0
01611760138693    201.0
01611760127233      NaN
01611920133629    140.0
                  ...  
56726035630249     29.0
56726030000001      0.0
56726035635008    211.0
56726035630322    169.0
56726035636188    206.0
56105610122713      0.0
56726525630793    195.0
56726525630298     65.0
56726525630348    127.0
56726520000001      0.0
567265256378

In [290]:
for col in grad_rates2.columns:
    print(col)

CharterYN
CohortStudents_GF
RegularHSGradCount_GF
RegularHSGradRate_GF
MetUCCSUCount_GF
MetUCCSURate_GF
DropoutCount_GF
DropoutRate_GF
CohortStudents_GM
RegularHSGradCount_GM
RegularHSGradRate_GM
MetUCCSUCount_GM
MetUCCSURate_GM
DropoutCount_GM
DropoutRate_GM
CohortStudents_GN
RegularHSGradCount_GN
RegularHSGradRate_GN
MetUCCSUCount_GN
MetUCCSURate_GN
DropoutCount_GN
DropoutRate_GN
CohortStudents_RA
RegularHSGradCount_RA
RegularHSGradRate_RA
MetUCCSUCount_RA
MetUCCSURate_RA
DropoutCount_RA
DropoutRate_RA
CohortStudents_RB
RegularHSGradCount_RB
RegularHSGradRate_RB
MetUCCSUCount_RB
MetUCCSURate_RB
DropoutCount_RB
DropoutRate_RB
CohortStudents_RD
RegularHSGradCount_RD
RegularHSGradRate_RD
MetUCCSUCount_RD
MetUCCSURate_RD
DropoutCount_RD
DropoutRate_RD
CohortStudents_RF
RegularHSGradCount_RF
RegularHSGradRate_RF
MetUCCSUCount_RF
MetUCCSURate_RF
DropoutCount_RF
DropoutRate_RF
CohortStudents_RH
RegularHSGradCount_RH
RegularHSGradRate_RH
MetUCCSUCount_RH
MetUCCSURate_RH
DropoutCount_RH
Dropo